# Complete RaTag Analysis Pipeline

This notebook orchestrates the complete analysis workflow:
1. **Run Preparation**: Load data, estimate S1 times, compute fields and transport properties
2. **X-ray Classification**: Identify and extract X-ray events for calibration
3. **Ion S2 Integration**: Integrate Ra recoil S2 signals and fit distributions
4. **Calibration & Recombination**: Use X-rays to calibrate energy scale and compute recombination

All processing is done through modular pipeline functions with functional programming principles.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from RaTag.datatypes import Run
from RaTag.config import IntegrationConfig, FitConfig
import RaTag.pipeline as pipeline
import RaTag.plotting as plotting

## 1. Define Run Parameters

In [ ]:
# Set data directory
base_dir = Path('/Volumes/KINGSTON/RaTag_data/RUN8_EL2350Vcm_5GSsec')

# Define run parameters
run8 = Run(
    root_directory = base_dir,
    run_id = "RUN8",
    el_field = 2375,            # V/cm
    target_isotope = "Th228",
    pressure = 2.0,             # bar
    temperature = 297,          # K
    sampling_rate = 5e9,        # Hz (5 GS/s for FastFrame)
    el_gap = 0.8,               # cm
    drift_gap = 1.4,            # cm
    width_s2 = 20,              # µs
    
    # Calibration constants
    W_value = 22.0,             # eV per e-ion pair (Xe @ 2 bar)
    E_gamma_xray = 12.3e3,      # eV (Th-228 X-ray energy)
)

## 2. Prepare Run

This step:
- Computes gas density from pressure/temperature
- Loads all measurement sets from directory structure
- Estimates S1 time for each set
- Computes drift/EL fields and transport properties

**Testing mode**: Set `nfiles` to process only a subset of files per set.

In [ ]:
# For testing: uncomment to limit to 5 files per set
# run8 = pipeline.prepare_run(run8, n_batches=1, batch_size=20, flag_plot=False, nfiles=5)

# Full run preparation
run8 = pipeline.prepare_run(run8, n_batches=1, batch_size=20, flag_plot=False)

In [ ]:
# Inspect prepared sets
for i, s in enumerate(run8.sets):
    print(f"{i}: {s.source_dir.name}")
    print(f"   t_s1: {s.metadata['t_s1']:.3f} µs")
    print(f"   Drift field: {s.drift_field:.1f} V/cm")
    print(f"   Drift time: {s.time_drift:.3f} µs")
    print()

### Optional: Manual Adjustment of S1 Times

If automatic S1 estimation is off for some sets, adjust manually:

In [ ]:
# Example: adjust S1 time for specific sets if needed
# run8.sets[2].metadata['t_s1'] = -3.32
# run8.sets[4].metadata['t_s1'] = -3.32

### Visualize S2 Windows

Check that S2 integration windows are correctly positioned:

In [ ]:
# Plot S2 windows for visual verification
plotting.plot_run_winS2(run8, ts2_tol=-2.7)

## 3. X-ray Event Classification

Classify X-ray events in the drift region for energy calibration.

**Testing mode**: Set `nfiles` to process only a subset of files.

In [ ]:
# Define integration configuration
xray_config = IntegrationConfig(
    bs_threshold = 2.0,        # mV
    max_area_s2 = 1e5,         # mV·µs
    min_s2_sep = 1.0,          # µs
    min_s1_sep = 1.0,          # µs
    n_pedestal = 200,          # samples
    ma_window = 10,            # samples
    dt = 2e-4,                 # µs
)

In [ ]:
# For testing: uncomment to process subset
# xray_results = pipeline.run_xray_classification(
#     run8, 
#     ts2_tol=-0.2, 
#     range_sets=slice(0, 2),  # Process only first 2 sets
#     config=xray_config,
#     nfiles=5                  # Only 5 files per set
# )

# Full X-ray classification
xray_results = pipeline.run_xray_classification(
    run8, 
    ts2_tol=-0.2, 
    config=xray_config
)

## 4. Ion S2 Integration

Integrate Ra recoil S2 signals and fit Gaussian distributions.

**Testing mode**: Set `nfiles` to process only a subset of files.

In [ ]:
# Define integration and fitting configurations
ion_int_config = IntegrationConfig(
    bs_threshold = 0.8,        # mV
    n_pedestal = 2000,         # samples
    ma_window = 9,             # samples
    dt = 2e-4,                 # µs
)

ion_fit_config = FitConfig(
    bin_cuts = (0, 10),        # mV·µs
    nbins = 100,
    exclude_index = 1,
)

In [ ]:
# For testing: uncomment to process subset
# ion_fitted = pipeline.run_ion_integration(
#     run8,
#     ts2_tol=-2.7,
#     range_sets=slice(0, 2),    # Process only first 2 sets
#     integration_config=ion_int_config,
#     fit_config=ion_fit_config,
#     nfiles=10                   # Only 10 files per set
# )

# Full ion S2 integration
ion_fitted = pipeline.run_ion_integration(
    run8,
    ts2_tol=-2.7,
    integration_config=ion_int_config,
    fit_config=ion_fit_config
)

### Visualize Ion S2 Fits

In [ ]:
# Plot histograms and fits for each set
for set_id, fit in ion_fitted.items():
    plotting.plot_hist_fit(fit, bin_cuts=(0, 10))

In [ ]:
# Plot S2 area vs drift field
plotting.plot_s2_vs_drift(fitted=ion_fitted, run=run8)

## 5. Calibration and Recombination Analysis

Use X-ray data to calibrate the energy scale and compute electron recombination fractions.

In [ ]:
# Run complete calibration analysis
calib_results, recomb_results = pipeline.run_calibration_analysis(
    run8,
    ion_fitted,
    xray_bin_cuts=(0.6, 20),
    xray_nbins=100,
    flag_plot=True
)

### Display Results

In [ ]:
print("Calibration Constants:")
print(f"  X-ray mean area (A_x): {calib_results.A_x_mean:.3f} mV·µs")
print(f"  Expected electrons (N_e_exp): {calib_results.N_e_exp:.1f}")
print(f"  Gain factor (g_S2): {calib_results.g_S2:.4f} mV·µs/electron")
print()
print("Recombination Results:")
for i, Ed in enumerate(recomb_results['drift_fields']):
    print(f"  {Ed:.0f} V/cm: r = {recomb_results['r'][i]:.3f} ± {recomb_results['dr'][i]:.3f}")

## 6. Export Results (Optional)

In [ ]:
# Save calibration results
results_dict = {
    'calibration': {
        'A_x_mean': calib_results.A_x_mean,
        'N_e_exp': calib_results.N_e_exp,
        'g_S2': calib_results.g_S2,
    },
    'recombination': recomb_results
}

# Uncomment to save
# import pickle
# with open(base_dir / f'{run8.run_id}_analysis_results.pkl', 'wb') as f:
#     pickle.dump(results_dict, f)
# print(f"Results saved to {base_dir / f'{run8.run_id}_analysis_results.pkl'}")